# Source Extration for 1-Photon Calcium Imaging Data

## Setting Parameters for Source Extraction and Motion Correction for 2p 

Specifically: How is this different to 2-Photon Imaging?

## `CNMFE` Algorithm and Correlation-PNR Plot for Resetting `Params` if necessary

## Exploring the Estimates Class

## Parallizing Source Extraction